# Input files pathes

In [1]:
import pandas as pd

#Input Files Pathes
by_path=input("Enter BY file path")
by_path= by_path[1:len(by_path)-1]


def str_to_raw(s):
    raw_map = {8:r'\b', 7:r'\a', 12:r'\f', 10:r'\n', 13:r'\r', 9:r'\t', 11:r'\v'}
    return r''.join(i if ord(i) > 32 else raw_map.get(ord(i), i) for i in s)

str_to_raw(by_path)


# Input SAP file path

sap_path=input("Enter SAP file path")
sap_path= sap_path[1:len(sap_path)-1]

def str_to_raw(s):
    raw_map = {8:r'\b', 7:r'\a', 12:r'\f', 10:r'\n', 13:r'\r', 9:r'\t', 11:r'\v'}
    return r''.join(i if ord(i) > 32 else raw_map.get(ord(i), i) for i in s)

str_to_raw(sap_path)

test_path = by_path.rsplit('\\',1)[0]+'\\test.xlsx' #To test results


# New columns names
new_columns_by = {'Generic Item Number':'generic_code_by' , 'Trade Item Number':'trade_code_by' , 'Total Qty':'quantity_by' ,'Lot/Batch':'supplier_batch_by','Location':'location_by','prtnum':'generic_code_by' , 'revlvl':'trade_code_by' , 'untqty(Available)':'available_quantity_by' ,'untqty(Hold)':'hold_quantity_by' ,'lotnum':'supplier_batch_by' }
new_columns_s4 = {'Scientific Item':'generic_code_sap' , 'Material':'trade_code_sap' , 'Unrestricted':'quantity_sap' ,'Supplier Batch':'supplier_batch_sap','Batch':'sap_batches'}                                                                                 

Enter BY file path "C:\Users\osama\Desktop\test_jeddah_stock\Full_Inventory_Status_Report (8) (1).xls"
Enter SAP file path "C:\Users\osama\Desktop\test_jeddah_stock\ZLOG010 W2C1-1000 27.1.2022.XLSX"


# *BY Data*

In [23]:
df_by=pd.read_excel(by_path)

In [3]:
df_by.columns

Index(['Generic Item Number', 'Trade Item Number', 'Description',
       'Customer Item Code', 'Avail Qty', 'Total Qty', 'UOM', 'Lot/Batch',
       'Production date', 'BBD', 'Inv Status', 'Custody', 'Location',
       'Supplier Name', 'Area Code', 'LPN', 'Hold', 'Warehouse', 'Client'],
      dtype='object')

In [4]:
df_by = df_by.rename(columns = new_columns_by)

if 'quantity_by' not in df_by:
    df_by['quantity_by']= df_by['available_quantity_by']+df_by['hold_quantity_by']
    
# Remove Staged Batches     
staged = ['1RSTG001','1RSTG002','1RSTG003','1RSTG004','1RSTG005','1RSTG006','1RSTG007','1RSTG008','1RSTGC','1RSTG','BULK01']   
if 'location_by' in df_by: 
    df_by = df_by[~df_by['location_by'].isin(staged)]

## Asessing Data and Cleaning Data

#### Remove Leading Zeros From Supplier Batches

In [21]:
df_by['supplier_batch_cleaned_by']= df_by['supplier_batch_by'].str.lstrip('0')

KeyError: 'supplier_batch_by'

#### Check Data Types

In [6]:
# Remove null before converting to integer
by_null = df_by[df_by['supplier_batch_cleaned_by'].isna()]
df_by= df_by[df_by['supplier_batch_cleaned_by'].notnull()]

df_by['generic_code_by'] = df_by['generic_code_by'].apply(int).astype(str)
df_by['trade_code_by'] = df_by['trade_code_by'].apply(int).astype(str)
df_by.head()

,generic_code_by,trade_code_by,Description,Customer Item Code,Avail Qty,quantity_by,UOM,supplier_batch_by,Production date,BBD,Inv Status,Custody,location_by,Supplier Name,Area Code,LPN,Hold,Warehouse,Client,supplier_batch_cleaned_by
0,4213150400200,4213150400203,DISPOSABLE COVERALL L QINGDAO ZHAOMING,137400128,540.0,540.0,EA,GF-5L3,2020-09-30,2025-08-30,Available,Hospital Supplies B,901048023204,NaN,1PALR,910006659073,NO,W100,MOH,GF-5L3
1,4213150400200,4213150400203,DISPOSABLE COVERALL L QINGDAO ZHAOMING,137400128,540.0,540.0,EA,GF-5L3,2020-09-30,2025-08-30,Available,Hospital Supplies B,901048090404,NaN,1PALR,910006659097,NO,W100,MOH,GF-5L3
2,4213150400200,4213150400203,DISPOSABLE COVERALL L QINGDAO ZHAOMING,137400128,180.0,180.0,EA,AMA-04RL,2020-09-30,2025-08-30,Available,Hospital Supplies B,901049016205,NaN,1PALR,910006659431,NO,W100,MOH,AMA-04RL
3,4213150400200,4213150400203,DISPOSABLE COVERALL L QINGDAO ZHAOMING,137400128,540.0,540.0,EA,IDEPL,2020-09-30,2025-08-30,Available,Hospital Supplies B,901053070408,NaN,1PALR,910006659264,NO,W100,MOH,IDEPL
4,4213150400200,4213150400203,DISPOSABLE COVERALL L QINGDAO ZHAOMING,137400128,540.0,540.0,EA,IDEPL,2020-09-30,2025-08-30,Available,Hospital Supplies B,901053097306,NaN,1PALR,910006659394,NO,W100,MOH,IDEPL


## Calculations

In [7]:
by_tableau=df_by.groupby(['generic_code_by','trade_code_by', 'supplier_batch_cleaned_by','supplier_batch_by'], as_index=False).sum()

#### Deal With Duplicated cleaned_batch/batch

In [8]:
df_by_dup = by_tableau[by_tableau.duplicated(['supplier_batch_cleaned_by','trade_code_by'],keep=False)]
df_by_dup['supplier_batch_cleaned_by'] =df_by_dup['supplier_batch_by'] 

by_tableau = by_tableau.drop_duplicates(['supplier_batch_cleaned_by','trade_code_by'],keep=False)
by_tableau=pd.concat([by_tableau,df_by_dup],axis=0)

<ipython-input-8-04c668e61ba7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_dup['supplier_batch_cleaned_by'] =df_by_dup['supplier_batch_by']


## Export BY File

In [9]:
null_by_path = by_path.rsplit('\\',1)[0]+'\\null_by.xlsx'
tableau_path = by_path.rsplit('\\',1)[0]+'\\tableau_by.xlsx'

by_null.to_excel(null_by_path)
by_tableau.to_excel(tableau_path)

# *S4 SAP*

In [10]:
df_sap = pd.read_excel(sap_path)

In [11]:
df_sap.columns

Index(['Customer Material Number', 'Several Customer Codes',
       'Old material no.', 'Scientific Item', 'Material', 'Material Number',
       'Plnt', 'Customer (GA)', 'SLoc', 'Batch', 'Supplier Batch', 'Cont./PO',
       'Several Supplier Batches', 'Manuf. Dte', 'SLED/BBD', 'Unrestricted',
       'BUn'],
      dtype='object')

In [12]:
df_sap = df_sap.rename(columns = new_columns_s4 )
df_sap.head()

,Customer Material Number,Several Customer Codes,Old material no.,generic_code_sap,trade_code_sap,Material Number,Plnt,Customer (GA),SLoc,sap_batches,supplier_batch_sap,Cont./PO,Several Supplier Batches,Manuf. Dte,SLED/BBD,quantity_sap,BUn
0,133060467,NaN,NaN,4.713150e+12,4.713150e+12,WIPE DISINFECTANT S625110MPCE PAL INTER,W2C1,120000.0,1000.0,2019/9130,2019/9130,5000000322,NaN,2021-04-01 00:00:00,2023-12-31 00:00:00,15400.0,EA
1,133060467,NaN,NaN,4.713150e+12,4.713150e+12,WIPE DISINFECTANT S625110MPCE PAL INTER,W2C1,120000.0,1000.0,21189145,21189145,5000000587,NaN,2021-07-17 00:00:00,2024-01-31 00:00:00,1200.0,EA
2,133060467,NaN,NaN,4.713150e+12,4.713150e+12,WIPE DISINFECTANT S625110MPCE PAL INTER,W2C1,120000.0,1000.0,21219145,21219145,5000000587,NaN,2021-07-17 00:00:00,2024-01-31 00:00:00,20400.0,EA
3,128290116,NaN,6.020302e+12,4.213161e+12,4.213161e+12,DAM SAFE FACE MASK 874421/9 NATIONAL MED,W2C1,120000.0,1000.0,300010M04,300010M04,COBFREE,NaN,2020-10-01 00:00:00,2025-09-30 00:00:00,38500.0,EA
4,128290116,NaN,6.020302e+12,4.213161e+12,4.213161e+12,DAM SAFE FACE MASK 874421/9 NATIONAL MED,W2C1,120000.0,1000.0,300010M105,300010M105,COBFREE,NaN,2020-10-01 00:00:00,2025-09-30 00:00:00,3200.0,EA


## Asessing and Cleaning

#### Remove Leading Zeros from Supplier Batches

In [13]:
df_sap['supplier_batch_cleaned_sap'] = df_sap['supplier_batch_sap'].str.lstrip('0')

#### Change Data Types

In [14]:
# Remove null before converting to integer
sap_null = df_sap[df_sap['supplier_batch_cleaned_sap'].isna()]
df_sap= df_sap[df_sap['supplier_batch_cleaned_sap'].notnull()]

df_sap['generic_code_sap'] = df_sap['generic_code_sap'].apply(int).astype(str)
df_sap['trade_code_sap'] = df_sap['trade_code_sap'].apply(int).astype(str)
df_sap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28203 entries, 0 to 28340
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Customer Material Number    27631 non-null  object 
 1   Several Customer Codes      28042 non-null  object 
 2   Old material no.            6817 non-null   float64
 3   generic_code_sap            28203 non-null  object 
 4   trade_code_sap              28203 non-null  object 
 5   Material Number             28203 non-null  object 
 6   Plnt                        28203 non-null  object 
 7   Customer (GA)               28203 non-null  float64
 8   SLoc                        28203 non-null  float64
 9   sap_batches                 28201 non-null  object 
 10  supplier_batch_sap          28203 non-null  object 
 11  Cont./PO                    28165 non-null  object 
 12  Several Supplier Batches    4028 non-null   object 
 13  Manuf. Dte                  229

## Calculations

In [15]:
sap_tableau=df_sap.groupby(['generic_code_sap','trade_code_sap','supplier_batch_cleaned_sap','supplier_batch_sap'], as_index=False).agg({
                          'sap_batches' : lambda x: "[%s]" % ', '.join(x.astype(str)), 'quantity_sap' : ['sum']})

sap_tableau.columns = sap_tableau.columns.droplevel(0)
sap_tableau.columns= ['generic_code_sap','trade_code_sap','supplier_batch_cleaned_sap','supplier_batch_sap','sap_batches','quantity_sap']

In [16]:
sap_tableau.head()

,generic_code_sap,trade_code_sap,supplier_batch_cleaned_sap,supplier_batch_sap,sap_batches,quantity_sap
0,4010151200000,4010151200001,219854701,219854701,[219854701],0.0
1,4010151200000,4010151200002,219175707,0219175707,[0219175707],5.0
2,4110291700000,4110291700002,9G2973,9G2973,[0000010307],2100.0
3,4110291700000,4110291700002,OB0693,OB0693,[OB0693],500.0
4,4110291700000,4110291700002,OD1384,OD1384,[0002339150],0.0


#### Deal with Duplicated cleaned_batch/batch duplicate

In [17]:
df_sap_dup = sap_tableau[sap_tableau.duplicated(['supplier_batch_cleaned_sap','trade_code_sap'],keep=False)]

df_sap_dup['supplier_batch_cleaned_sap'] =df_sap_dup['supplier_batch_sap'] 

sap_tableau = sap_tableau.drop_duplicates(['supplier_batch_cleaned_sap','trade_code_sap'],keep=False)
sap_tableau=pd.concat([sap_tableau,df_sap_dup],axis=0)

<ipython-input-17-d2d559324e04>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sap_dup['supplier_batch_cleaned_sap'] =df_sap_dup['supplier_batch_sap']


## Export SAP Files

In [18]:
sap_tableau.to_excel(r"C:\Users\osama\Documents\nupco\stock adjustment\sap_grouped.xls")
df_sap_dup.to_excel(r"C:\Users\osama\Documents\nupco\stock adjustment\sap_duplicates.xls")

In [19]:
null_sap_path = by_path.rsplit('\\',1)[0]+'\\null_sap.xlsx'
tableau_path = by_path.rsplit('\\',1)[0]+'\\tableau_sap.xlsx'

by_null.to_excel(null_sap_path)
sap_tableau.to_excel(tableau_path)